In [1]:
import numpy as np
from astropy import conf as config
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import QTable, join, vstack
from astroquery.simbad import Simbad

config.max_width = -1

Simbad.clear_cache()
Simbad.reset_votable_fields()
Simbad.add_votable_fields(
    "otype", "flux"
)  # , 'mesDiameter')#, 'mesDistance') # allfluxes

columns_to_remove = [
    "coo_err_min",
    "coo_err_angle",
    "coo_wavelength",
    "coo_bibcode",
    "coo_err_maj",
    "bibcode",
    "flux_err",
    "flux_err_prec",
    "flux_prec",
    "oid_ref",
    "qual",
    "mesdiameter.bibcode",
    "mesdiameter.diameter_prec",
    "mesdiameter.error",
    "mesdiameter.error_prec",
    "mesdiameter.method",
    "mesdiameter.qual",
    "mesdiameter.mespos",
    "mesdistance.bibcode",
    "mesdistance.dist_prec",
    "mesdistance.mespos",
    "mesdistance.method",
    "mesdistance.minus_err",
    "mesdistance.minus_err_prec",
    "mesdistance.plus_err",
    "mesdistance.plus_err_prec",
    "mesdistance.qual",
]

criteria = "otype != 'err'"

In [2]:
vega = SkyCoord(
    ra=18 * u.hour + 36 * u.minute + 56.33635 * u.second,
    dec=38 * u.deg + 47 * u.arcmin + 1.28 * u.arcsec,
)
omc = SkyCoord(ra=5 * u.hour + 35.3 * u.minute, dec=-(5 * u.deg + 23 * u.arcmin))

radius = 1 * u.deg

source = omc

Simbad.reset_votable_fields()
Simbad.add_votable_fields("otype", "flux")
basic_result = QTable(Simbad.query_region(source, radius=radius, criteria=criteria))

Simbad.reset_votable_fields()
Simbad.add_votable_fields("mesDiameter")
diameter_result = QTable(Simbad.query_region(source, radius=radius, criteria=criteria))

Simbad.reset_votable_fields()
Simbad.add_votable_fields("mesDistance")
distance_result = QTable(Simbad.query_region(source, radius=radius, criteria=criteria))

Simbad.reset_votable_fields()
Simbad.add_votable_fields("plx_value")
parallax_result = QTable(Simbad.query_region(source, radius=radius, criteria=criteria))

for table in [basic_result, diameter_result, distance_result, parallax_result]:
    for colname in columns_to_remove:
        if colname in table.colnames:
            table.remove_column(colname)
query_result = join(
    basic_result, diameter_result, keys=["main_id", "ra", "dec"], join_type="left"
)  # include all basic, and diam when available
query_result = join(
    query_result, distance_result, keys=["main_id", "ra", "dec"], join_type="left"
)
query_result = join(
    query_result, parallax_result, keys=["main_id", "ra", "dec"], join_type="left"
)

# print(query_result)

vegarow = query_result["main_id"] == "* alf Lyr"
print(query_result[vegarow])

main_id  ra dec otype filter flux mesdiameter.diameter mesdiameter.filter mesdiameter.unit mesdistance.dist mesdistance.unit plx_value
        deg deg                                                                                                                 mas   
------- --- --- ----- ------ ---- -------------------- ------------------ ---------------- ---------------- ---------------- ---------


In [3]:
omc = SkyCoord(ra=5 * u.hour + 35.3 * u.minute, dec=-(5 * u.deg + 23 * u.arcmin))
vega = SkyCoord(
    ra=18 * u.hour + 36 * u.minute + 56.33635 * u.second,
    dec=38 * u.deg + 47 * u.arcmin + 1.28 * u.arcsec,
)
radius = 1 * u.deg

magnitude_cutoff = 7

# query_result = QTable(Simbad.query_region(vega, radius=radius, criteria=criteria))
# for colname in columns_to_remove:
#     if colname in query_result.colnames:
#         query_result.remove_column(colname)


# basic
query_result.rename_column("otype", "object_type")
query_result.rename_column("main_id", "id")
query_result["ra"] = query_result["ra"].round(5)
query_result["dec"] = query_result["dec"].round(5)

# diameter
diameter_colnames = [n for n in query_result.colnames if "mesdiameter" in n]
query_result.rename_columns(
    diameter_colnames, [f"diameter{i}" for i in ["", ".filter", ".unit"]]
)
query_result["diameter"] = query_result["diameter"].data.filled(np.nan) * [
    u.Unit(i) for i in query_result["diameter.unit"].data.filled("km")
]
query_result["diameter"] = [i.to(u.R_sun) for i in query_result["diameter"]]
query_result["diameter"] = query_result["diameter"].round(3)
query_result["diameter.filter"] = query_result["diameter.filter"].filled("")
query_result.remove_column("diameter.unit")

# magnitude
query_result = query_result[np.argsort(query_result["flux"])]
query_result.rename_columns(("filter", "flux"), ("magnitude.filter", "magnitude"))
query_result = query_result[query_result["magnitude.filter"] == "V"]
query_result.remove_column("magnitude.filter")
query_result = query_result[query_result["magnitude"] <= magnitude_cutoff]

# # distance
query_result["distance"] = query_result["mesdistance.dist"].round(2).data.filled(
    np.nan
) * [u.Unit(i) for i in query_result["mesdistance.unit"].filled("pc")]
query_result.remove_columns(["mesdistance.dist", "mesdistance.unit"])

query_result.rename_column("plx_value", "parallax")


print(query_result)
vegarow = query_result["id"] == "* alf Lyr"

print(query_result[vegarow])

     id        ra      dec    object_type magnitude diameter diameter.filter parallax  distance
              deg      deg                           solRad                    mas             
----------- -------- -------- ----------- --------- -------- --------------- -------- ---------
  NGC  1980    83.81   -5.924         OpC       2.5      nan                    2.586  520.0 pc
  NGC  1980    83.81   -5.924         OpC       2.5      nan                    2.586  377.0 pc
  NGC  1980    83.81   -5.924         OpC       2.5      nan                    2.586  483.0 pc
  NGC  1980    83.81   -5.924         OpC       2.5      nan                    2.586  297.0 pc
  NGC  1980    83.81   -5.924         OpC       2.5      nan                    2.586  0.39 kpc
  NGC  1980    83.81   -5.924         OpC       2.5      nan                    2.586  550.0 pc
  NGC  1980    83.81   -5.924         OpC       2.5      nan                    2.586  316.0 pc
  NGC  1980    83.81   -5.924         Op

In [4]:
np.unique(query_result["diameter.filter"].data)

array([''], dtype=object)

In [5]:
unique_ids = np.unique(query_result["id"].data, return_index=True)[0]
query_result.add_index("id")

deduplicated_result = []

for uid in unique_ids:
    item_duplicated = query_result.loc[uid]
    item_unique = QTable()
    item_unique["id"] = [uid]
    item_unique["ra"] = [item_duplicated["ra"].mean()]
    item_unique["dec"] = [item_duplicated["dec"].mean()]
    item_unique["magnitude"] = [
        item_duplicated["magnitude"].mean()
    ]  # all measured w/ same filter (V)
    item_unique["distance"] = [item_duplicated["distance"].mean()]

    object_type_options = np.unique(item_duplicated["object_type"])
    if len(object_type_options) == 1:
        item_unique["object_type"] = object_type_options
    else:
        raise Exception(f"Item {uid} has multiple object types: {object_type_options}")

    diameter_filter_options = np.unique(item_duplicated["diameter.filter"])
    if len(diameter_filter_options) == 1:  # all diameters measured with the same filter
        item_unique["diameter.filter"] = diameter_filter_options[0]
        item_unique["diameter"] = [item_duplicated["diameter"].mean()]
    else:
        given_filter_options = [o for o in diameter_filter_options if len(o) > 0]
        if len(given_filter_options) == 1:  # ignore the unknown filter options
            item_unique["diameter.filter"] = given_filter_options
            diameter_rows = (
                item_duplicated["diameter.filter"] == given_filter_options[0]
            )
            item_unique["diameter"] = [
                (item_duplicated["diameter"][diameter_rows]).mean()
            ]
        else:
            raise Exception(
                f"Item {uid} has diameters measured in multiple filters: {given_filter_options}"
            )

    deduplicated_result.append(item_unique)

deduplicated_query = vstack(deduplicated_result)


print(deduplicated_query)

TableMergeError: The 'object_type' columns have incompatible types: ['object', 'str96', 'str96', 'object', 'object', 'object', 'object', 'str96', 'object', 'object', 'object', 'object', 'str32', 'str32', 'object', 'object', 'object', 'object', 'object', 'object', 'object', 'object']

In [ ]:
Simbad.list_votable_fields().show_in_notebook()

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_renderer=None, default_render…

In [ ]:
Simbad.list_columns("mesdistance").show_in_notebook()

DataGrid(auto_fit_params={'area': 'all', 'padding': 30, 'numCols': None}, corner_renderer=None, default_render…